# 🍏 Fun & Fit 헬스 어드바이저 에이전트 자습서 🍎

**Azure AI Foundry** SDK를 사용하여 재미있게(하지만 신중하게!) 건강 및 피트니스 도우미를 만드는 **Fun & Fit Health Advisor 에이전트** 자습서에 오신 것을 환영합니다. 다음을 수행하게 됩니다:

1. **azure-ai-projects**를 사용하여 프로젝트를 **초기화**합니다.
2. 일반적인 건강 및 영양 조언을 제공하는 데 특화된 **에이전트**를 만듭니다(면책 조항 포함!).
3. 피트니스, 영양 및 일반 건강 주제에 대한 **대화 관리**.
4. **OpenTelemetry**로 **쇼케이스 로깅 및 추적**.
5. 도구, 안전 고지 사항 및 기본 모범 사례를 통합하는 방법을 **시연**합니다.

### ⚠️ 중요 의료 고지 사항 ⚠️
> **이 노트에서 제공하는 건강 정보는 일반적인 교육 및 오락 목적으로만 사용되며 전문적인 의학적 조언, 진단 또는 치료를 대신할 수 없습니다.** 건강 상태와 관련하여 궁금한 점이 있으면 항상 의사 또는 기타 자격을 갖춘 의료 제공자의 조언을 구하세요. 이 노트에서 읽거나 받은 내용 때문에 전문적인 의학적 조언을 무시하거나 의사의 진료를 미루지 마세요.



## 전제 조건

[the notebooks in introduction](../../1-introduction/3-quick_start.ipynb)를 완료하세요.

## 시작하기
각 셀을 노트와 다이어그램을 통해 재미있게 안내해 드리겠습니다. 시작하죠!

<img src="./seq-diagrams/1-basics.png" width="30%"/>




## 1. 초기 설정
필요한 라이브러리를 가져오고, 환경 변수를 로드하고, 에이전트 관련 모든 작업을 수행할 수 있도록 **AIProjectClient**를 초기화하는 것으로 시작하겠습니다. 🎉


In [ ]:
# Import required libraries
import os  # For environment variables and path operations
import time  # For adding delays if needed
from pathlib import Path  # For cross-platform path handling

# Import Azure and utility libraries
from dotenv import load_dotenv  # For loading environment variables from .env file
from azure.identity import DefaultAzureCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Main client for AI Projects
from azure.ai.projects.models import MessageTextContent  # For handling message content

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv(parent_dir / '.env')  # Load environment variables from .env file

# Initialize the AI Project Client using connection string from environment variables
try:
    project_client = AIProjectClient.from_connection_string(
        # Use default Azure credentials for authentication
        credential=DefaultAzureCredential(),
        # Get the project connection string from environment variables
        conn_str=os.environ.get("PROJECT_CONNECTION_STRING")
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    # Print error message if client initialization fails
    print(f"❌ Error initializing project client: {str(e)}")

## 2. 펀앤핏 헬스 어드바이저 에이전트 만들기 🏋️

일반 건강 및 웰니스에 특화된 에이전트를 만들겠습니다. 에이전트 지침에 면책 조항을 명시적으로 언급하여 안전을 잊지 않도록 할 것입니다! 또한 에이전트에게 일반적인 피트니스와 식이요법 팁에 초점을 맞추고 항상 전문가의 조언을 구하도록 권장합니다.


In [ ]:
def create_health_advisor_agent():
    """Create a health advisor agent with disclaimers and basic instructions."""
    try:
        # Get the model name from environment variables, default to gpt-4o-mini if not set
        model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

        # Create a new agent using the AIProjectClient
        # The agent will use the specified model and follow the given instructions
        agent = project_client.agents.create_agent(
            model=model_name,
            name="fun-fit-health-advisor",
            # Define the agent's behavior and responsibilities
            instructions="""
            You are a friendly AI Health Advisor.
            You provide general health, fitness, and nutrition information, but always:
            1. Include medical disclaimers.
            2. Encourage the user to consult healthcare professionals.
            3. Provide general, non-diagnostic advice around wellness, diet, and fitness.
            4. Clearly remind them you're not a doctor.
            5. Encourage safe and balanced approaches to exercise and nutrition.
            """
        )
        # Log success and return the created agent
        print(f"🎉 Created health advisor agent, ID: {agent.id}")
        return agent
    except Exception as e:
        # Handle any errors during agent creation
        print(f"❌ Error creating agent: {str(e)}")
        return None

# Create an instance of our health advisor agent
health_advisor = create_health_advisor_agent()

## 3. 건강 대화 관리하기 💬
대화(또는 *스레드*)는 건강 주제에 대한 사용자의 메시지와 상담원의 응답을 저장하는 곳입니다. 건강 및 피트니스 Q&A 전용 스레드를 새로 만들어 보겠습니다.


In [ ]:
# Function to create a new conversation thread for health discussions
def start_health_conversation():
    """Create a new thread for health & fitness discussions."""
    try:
        # Create a new empty thread using the project client
        # A thread stores the back-and-forth messages between user and agent
        thread = project_client.agents.create_thread()
        
        # Print success message with the thread ID for reference
        print(f"📝 Created a new conversation thread, ID: {thread.id}")
        
        # Return the created thread object so we can use it later
        return thread
    except Exception as e:
        # If thread creation fails, print error and return None
        print(f"❌ Error creating thread: {str(e)}")
        return None

# Initialize a new conversation thread that we'll use for our health Q&A
health_thread = start_health_conversation()

## 4. 건강 및 피트니스 관련 질문하기 🏃‍♂️
일반적인 건강 관련 질문에 대한 사용자의 메시지를 생성합니다. 예를 들어, **“체질량지수(BMI)는 어떻게 계산하나요?”** 또는 **“활동적인 라이프스타일을 위한 균형 잡힌 식단은 무엇인가요?”**와 같은 질문이 있습니다. 건강 어드바이저 상담원이 항상 고지 사항을 기억하면서 답변할 수 있도록 하겠습니다!


In [4]:
def ask_health_question(thread_id, user_question):
    """Add a user message to the conversation thread.
    
    Args:
        thread_id: ID of the conversation thread
        user_question: The health/fitness question from the user
        
    Returns:
        Message object if successful, None if error occurs
    """
    try:
        # Create a new message in the thread from the user's perspective
        # The role="user" indicates this is a user message (vs assistant)
        return project_client.agents.create_message(
            thread_id=thread_id,
            role="user", 
            content=user_question
        )
    except Exception as e:
        print(f"❌ Error adding user message: {e}")
        return None

def process_thread_run(thread_id, agent_id):
    """Ask the agent to process the thread and generate a response.
    
    Args:
        thread_id: ID of the conversation thread
        agent_id: ID of the health advisor agent
        
    Returns:
        Run object if successful, None if error occurs
    """
    try:
        # Create a new run to have the agent process the thread
        run = project_client.agents.create_run(
            thread_id=thread_id,
            assistant_id=agent_id
        )

        # Poll the run status until completion or error
        # Status can be: queued, in_progress, requires_action, completed, failed
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)  # Wait 1 second between status checks
            run = project_client.agents.get_run(
                thread_id=thread_id,
                run_id=run.id
            )

        print(f"🤖 Run completed with status: {run.status}")
        return run
    except Exception as e:
        print(f"❌ Error processing thread run: {str(e)}")
        return None

def view_thread_messages(thread_id):
    """Display all messages in the conversation thread in chronological order.
    
    Args:
        thread_id: ID of the conversation thread to display
    """
    try:
        # Get all messages in the thread
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation so far (oldest to newest):")
        
        # Loop through messages in reverse order to show oldest first
        for m in reversed(messages.data):
            if m.content:
                # Extract the text content from the message
                # We only handle text messages for now (not images etc)
                last_content = m.content[-1]
                if isinstance(last_content, MessageTextContent):
                    print(f"{m.role.upper()}: {last_content.text.value}\n")
        print("-----------------------------------\n")
    except Exception as e:
        print(f"❌ Error viewing thread: {str(e)}")

### 예제 쿼리
이제 몇 가지 간단한 질문을 통해 상담원의 고지 사항과 일반적인 건강 관련 질문을 어떻게 처리하는지 확인해 보겠습니다. 활동적인 라이프스타일을 위한 **체질량지수**와 **균형 잡힌 식사**에 대해 물어보겠습니다.


In [ ]:
# First verify that we have valid agent and thread objects before proceeding
if health_advisor and health_thread:
    # 1) Ask about BMI calculation and interpretation
    # This demonstrates how the agent handles technical health questions
    msg1 = ask_health_question(health_thread.id, "How do I calculate my BMI, and what does it mean?")
    # Process the BMI question and wait for agent's response
    run1 = process_thread_run(health_thread.id, health_advisor.id)

    # 2) Ask about personalized meal planning
    # This shows how the agent provides customized nutrition advice
    msg2 = ask_health_question(health_thread.id, "Can you give me a balanced meal plan for someone who exercises 3x a week?")
    # Process the meal plan question and wait for agent's response
    run2 = process_thread_run(health_thread.id, health_advisor.id)

    # Display the full conversation history to see both Q&As
    # This will show the agent's responses including any health disclaimers
    view_thread_messages(health_thread.id)
else:
    # Error handling if agent or thread initialization failed
    print("❌ Could not run example queries because agent or thread is None.")

## 5. 정리하기 🧹
완료된 후 서비스에서 에이전트를 제거하려면 아래 코드로 제거할 수 있습니다. (프로덕션 환경에서는 상태 저장 경험을 위해 에이전트를 계속 유지할 수도 있습니다!)

In [ ]:
# Function to clean up and delete the agent when we're done
def cleanup(agent):
    # Only attempt cleanup if we have a valid agent
    if agent:
        try:
            # Delete the agent using the project client
            project_client.agents.delete_agent(agent.id)
            # Print confirmation message with the agent name
            print(f"🗑️ Deleted health advisor agent: {agent.name}")
        except Exception as e:
            # Handle any errors that occur during deletion
            print(f"Error cleaning up agent: {e}")
    else:
        # If no agent was provided, inform the user
        print("No agent to clean up.")

# Call cleanup function to delete our health advisor agent
cleanup(health_advisor)

# 축하합니다! 🏆
**펀앤핏 헬스 어드바이저**를 성공적으로 구축했습니다:
1. 기본적인 건강 및 피트니스 질문에 **응답**합니다.
2. 안전하고 전문적인 상담을 장려하기 위해 **면책 조항**을 사용합니다.
3. 일반적인 식단 및 건강 정보를 **제공**합니다.
4. **Azure AI Foundry** 모듈의 시너지를 활용하여 대화를 강화합니다.

## 다음 단계
- 보다 전문적인 정보를 제공하기 위한 고급 도구(예: **FileSearchTool** 또는 **CodeInterpreterTool**)를 추가하는 것을 고려해보세요.
- **azure-ai-evaluation**으로 AI의 성능을 평가하세요!
- 더 심층적인 인사이트를 얻으려면 **OpenTelemetry** 또는 Azure Monitor를 추가하세요.
- 고급 계산 또는 직접 데이터 분석과 같은 작업을 처리하려면 **함수 호출**을 통합하세요.

#### [2-code_interpreter.ipynb](2-code_interpreter.ipynb)로 이동합니다.

행복한 (그리고 건강한) 코딩되세요! 💪